# December 04, 2021

https://adventofcode.com/2021/day/4

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.__version__

'1.26.4'

# Read Data

In [3]:
balls = pd.read_csv("../data/2021/04.txt", sep=",", nrows=1, header=None)
balls = list( balls.iloc[0] )

In [4]:
with open("../data/2021/04.txt", "r") as f:
    data = f.read()

nlines = len(data.split("\n"))
nboards = int( (nlines-1)/6 )

In [5]:
nboards

100

In [6]:
def read_boards():
    boards = []
    for i in range(nboards):
        bd = pd.read_csv( "../data/2021/04.txt", sep="\s+", nrows=5, header = None, skiprows = i*6+2, skip_blank_lines=False )
        boards.append(bd)
    return boards

boards = read_boards()

# Part 1

In [7]:
boards[0]

,0,1,2,3,4
0,22,59,7,10,6
1,33,36,96,55,23
2,13,85,18,29,28
3,75,46,83,73,58
4,34,40,87,56,98


In [8]:
bd = boards[0]
pd.concat( [bd.apply(min, axis=1), bd.apply(min, axis=0)] )

0     6
1    23
2    13
3    46
4    34
0    13
1    36
2     7
3    10
4     6
dtype: int64

In [9]:
pd.__version__

'2.2.1'

In [10]:
# user 1000+num to indicate a found number.
# victory means min in a row or col is 1000+
# assumes all numbers between 0 and 999

MARKER = 1000
def call_num_board(bd, num):
    nc = bd.shape[1]
    for c in range(nc):
        check = [r for r,x in enumerate(bd[c]) if x == num]
        if len(check) > 0:
            bd.iloc[check[0], c] = MARKER + num
            exit
            return bd, True

    return bd, False

def check_win_board(bd):
    # get the min value for each row and for each column
    # if at least one min >= MARKER, then that row/col wins
    cond = max( pd.concat( [bd.apply(min, axis=1), bd.apply(min, axis=0)] ) )
    return cond >= MARKER

def call_num(boards, num):
    win = False
    for bidx, bd in enumerate(boards):
        bd, hit = call_num_board(bd, num)
        if hit:
            win = check_win_board(bd)
            if win:
                break

    if win:
        print("board", bidx, "wins!")
        display(bd)
        return bidx
    
    return None

def play_game(boards, balls):
    for ball in balls:
        #print("next ball:", ball)
        win_idx = call_num(boards, ball)
        if win_idx is not None:
            break

    if win_idx is None:
        print("no winner yet")
        return None
    else:
        print("wtg", win_idx, "!")
        return boards[win_idx], ball
    
def score_board( bd, ball ):
    nums = bd.values.flatten().tolist()
    return ball * sum( [x for x in nums if x < MARKER] )

In [11]:
boards = read_boards()
winner, last_ball = play_game(boards, balls)

board 69 wins!


,0,1,2,3,4
0,51,1000,14,7,1053
1,63,1036,1048,81,1084
2,1050,22,11,88,1006
3,83,99,21,31,1091
4,86,17,72,42,1094


wtg 69 !


In [12]:
score_board( winner, last_ball )

71708

# Part 2

In [13]:
def call_num2(boards, num):
    winners = []

    for bidx, bd in enumerate(boards):
        bd, hit = call_num_board(bd, num)

        # check if the board wins
        if hit:
            if check_win_board(bd):
                winners.append(bidx)

    # return ids of all winners
    return winners

def find_loser(boards, balls):
    for i , ball in enumerate(balls):
        #print("next ball:", ball)
        winners = call_num2(boards, ball)

        winners.sort(reverse=True)
        for bidx in winners:
            del boards[bidx]

        if len(boards) == 1:
            return boards, i
        
        assert len(boards) > 0, "tie for worst?!"

    print("what happened?")
    return None, None

def score_board( bd, ball ):
    nums = bd.values.flatten().tolist()
    return ball * sum( [x for x in nums if x < MARKER] )

In [14]:
boards = read_boards()
loser, ball_id = find_loser(boards, balls)

In [15]:
loser

[      0     1     2     3     4
 0  1021    97    92    42  1060
 1  1011  1065    98  1095  1029
 2     2  1049  1075    20  1074
 3    56  1040  1078  1066    81
 4  1068  1004  1046  1077    61]

In [16]:
loser[0]

,0,1,2,3,4
0,1021,97,92,42,1060
1,1011,1065,98,1095,1029
2,2,1049,1075,20,1074
3,56,1040,1078,1066,81
4,1068,1004,1046,1077,61


In [17]:
loser, last_ball = play_game( loser, balls[(ball_id+1):] )

board 0 wins!


,0,1,2,3,4
0,1021,1097,1092,42,1060
1,1011,1065,98,1095,1029
2,1002,1049,1075,20,1074
3,56,1040,1078,1066,81
4,1068,1004,1046,1077,61


wtg 0 !


In [18]:
score_board(loser, last_ball)

34726

In [19]:
42+98+20+81+61+56

358

In [20]:
358*97

34726